In [1]:
import pandas as pd
import random
from copy import deepcopy

# Cargar datos
def cargar_disponibilidad(csv_path):
    return pd.read_csv(csv_path, index_col="MentorID")

# Generar bloques de 2h disponibles para cada mentor
def obtener_bloques_validos(df):
    bloques = {}
    for mentor, row in df.iterrows():
        bloques[mentor] = []
        for i in range(len(row) - 1):
            if row.iloc[i] == 1 and row.iloc[i+1] == 1:
                bloques[mentor].append((i, i+1))
    return bloques

# Generar una solución inicial aleatoria
def solucion_inicial(bloques_validos):
    return {mentor: random.choice(bloques) for mentor, bloques in bloques_validos.items() if bloques}

# Calcular choques (costo)
def calcular_choques(solucion):
    contador = {}
    for bloque in solucion.values():
        for slot in bloque:
            contador[slot] = contador.get(slot, 0) + 1
    return sum(v - 1 for v in contador.values() if v > 1)

# Vecindad: cambiar un bloque de un mentor
def generar_vecindad(solucion, bloques_validos):
    vecinos = []
    for mentor in solucion:
        for nuevo_bloque in bloques_validos[mentor]:
            if nuevo_bloque != solucion[mentor]:
                nuevo = deepcopy(solucion)
                nuevo[mentor] = nuevo_bloque
                vecinos.append(nuevo)
    return vecinos

# Búsqueda local
def busqueda_local(df):
    bloques_validos = obtener_bloques_validos(df)
    solucion = solucion_inicial(bloques_validos)
    mejor_costo = calcular_choques(solucion)

    while True:
        vecinos = generar_vecindad(solucion, bloques_validos)
        mejor_vecino = solucion
        for vecino in vecinos:
            costo = calcular_choques(vecino)
            if costo < mejor_costo:
                mejor_costo = costo
                mejor_vecino = vecino
        if mejor_vecino == solucion:  # sin mejora
            break
        solucion = mejor_vecino
        if mejor_costo == 0:
            break

    return solucion, mejor_costo


In [2]:
df = cargar_disponibilidad("mentorAvailability.csv")
solucion_final, choques = busqueda_local(df)

print("Asignación final:")
for mentor, bloque in solucion_final.items():
    print(f"{mentor}: Slots {bloque[0]} y {bloque[1]}")
print(f"Choques: {choques}")


Asignación final:
M01: Slots 1 y 2
M02: Slots 0 y 1
M03: Slots 3 y 4
M04: Slots 6 y 7
M05: Slots 4 y 5
M06: Slots 4 y 5
M07: Slots 8 y 9
M08: Slots 7 y 8
M09: Slots 0 y 1
M10: Slots 2 y 3
M12: Slots 0 y 1
M13: Slots 8 y 9
M14: Slots 7 y 8
M15: Slots 4 y 5
M16: Slots 0 y 1
M17: Slots 1 y 2
M18: Slots 4 y 5
M19: Slots 7 y 8
Choques: 26
